In [1]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import warnings
warnings.filterwarnings("ignore")

In [2]:
uploaded = files.upload()

Saving amazon.csv to amazon.csv


In [69]:
#read csv file
df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
#dfReview for review title and review content
dfText = df[['product_name','about_product','category']]

In [70]:
for column in range(2):
  dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
  dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))
dfText['category'] = dfText['category'].apply(lambda x: x.split('|',1)[0])

#combine product_name and about_product
dfText['combined'] = dfText['product_name'] + ' ' + dfText['about_product']

In [71]:
#tokenize, build vocab, and add paddings on product_name
tokenizer = Tokenizer(num_words=2000,split=' ')
tokenizer.fit_on_texts(dfText['combined'].values)
text = tokenizer.texts_to_sequences(dfText['combined'].values)
text = pad_sequences(text)

#category
category = pd.get_dummies(dfText['category']).values

print(text.shape)
print(category.shape)

(1465, 424)
(1465, 9)


In [72]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten, Reshape, SpatialDropout1D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [73]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(text, category, test_size=0.2, random_state=42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

#apply Bidirectional LSTM
model = Sequential()
model.add(Embedding(2000, 128, input_length = 424))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

#Train the model for 5 epoches with embedding matrix you obtained earlier.
#print training loss and training accuracy for each epoch
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test,Y_test))  


(1172, 424) (1172, 9)
(293, 424) (293, 9)
Epoch 1/10
37/37 [==============================] - 95s 2s/step - loss: 1.4326 - accuracy: 0.3464 - val_loss: 1.2116 - val_accuracy: 0.3379
Epoch 2/10
37/37 [==============================] - 91s 2s/step - loss: 1.2173 - accuracy: 0.3584 - val_loss: 1.2153 - val_accuracy: 0.4096
Epoch 3/10
37/37 [==============================] - 91s 2s/step - loss: 1.0802 - accuracy: 0.5742 - val_loss: 0.7732 - val_accuracy: 0.6724
Epoch 4/10
37/37 [==============================] - 91s 2s/step - loss: 1.0367 - accuracy: 0.5776 - val_loss: 0.9452 - val_accuracy: 0.6314
Epoch 5/10
37/37 [==============================] - 91s 2s/step - loss: 0.5970 - accuracy: 0.8200 - val_loss: 0.5583 - val_accuracy: 0.8157
Epoch 6/10
37/37 [==============================] - 91s 2s/step - loss: 0.3090 - accuracy: 0.9036 - val_loss: 0.4928 - val_accuracy: 0.8362
Epoch 7/10
37/37 [==============================] - 90s 2s/step - loss: 0.2223 - accuracy: 0.9292 - val_loss: 0.4401 -